In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import calendar
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  

# 2. Rebuild missing data

In [2]:
localidadCodDictionaryNuse = {1:'USAQUEN',
                              2:'CHAPINERO',
                              3:'SANTA FE',
                              4:'SAN CRISTOBAL',
                              5:'USME',
                              6:'TUNJUELITO',
                              7:'BOSA',
                              8:'KENNEDY',
                              9:'FONTIBON',
                              10:'ENGATIVA',
                              11:'SUBA',
                              12:'BARRIOS UNIDOS',
                              13:'TEUSAQUILLO',
                              14:'LOS MARTIRES',
                              15:'ANTONIO NARIÑO',
                              16:'PUENTE ARANDA',
                              17:'CANDELARIA',
                              18:'RAFAEL URIBE URIBE',
                              19:'CIUDAD BOLIVAR',
                              20:'SUMAPAZ',
                              99:'SIN LOCALIZACION'}

## Methods to rebuild

In [3]:
import import_ipynb
import ws_address
from selenium.common.exceptions import TimeoutException
import re
import unidecode

importing Jupyter notebook from ws_address.ipynb


In [4]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [5]:
tags = ["Dirección ingresada: ","Dirección encontrada: ","Tipo dirección: ","Código postal: ","Sector catastral: ",
        "UPZ: ","Localidad: ","Latitud: ","Longitud: ","CHIP: "]
def parse_address_ws(ws_result):
    location = {}
    for idx in range(len(tags)-1):
        location[tags[idx].replace(': ','')] = find_between(ws_result,tags[idx],tags[idx+1])
    return location

In [6]:
def assign_upz(original_df,index,UPZ_ws_field):
    original_df.at[index,'COD_UPZ'] = find_between(UPZ_ws_field, '(', ')')
    original_df.at[index,'UPZ'] = find_between(UPZ_ws_field, '', ' (')

In [7]:
def get_cod_localidad(localidad_name):
    return [key  for (key, value) in localidadCodDictionaryNuse.items() if value == localidad_name][0]

In [8]:
def rebuild_location_in_nuse(original_df, index, driver):
    address = original_df.at[index,'STR_DIRECCION_INCIDENTE']
    print(address)
    result_ws = ws_address.web_scrap_address(driver,address)
    ws_address.delete_address(driver,address)

    if result_ws != "Not found":
        parsed_result = parse_address_ws(result_ws)
        print(parsed_result)
        if parsed_result["Dirección ingresada"] != address:
            return "Error loading address"
        else:            
            original_df.at[index,'LATITUD'] = float(parsed_result['Latitud'])
            original_df.at[index,'LONGITUD'] = float(parsed_result['Longitud'])
            parsed_localidad = parsed_result['Localidad']
            if parsed_localidad == 'ANTONIO NARIÑO':
                original_df.at[index,'LOCALIDAD'] = parsed_localidad
            else:
                original_df.at[index,'LOCALIDAD'] = unidecode.unidecode(parsed_localidad)
            original_df.at[index,'COD_LOCALIDAD'] = int(get_cod_localidad(original_df.at[index,'LOCALIDAD']))
            original_df.at[index,'SEC_CATASTRAL'] = parsed_result['Sector catastral']
            assign_upz(original_df,index,parsed_result['UPZ'])
            return "Rebuilt"
    else:
        return "Not processed"


In [9]:
def rebuild_address_in_nuse(original_df, index):
    log_text = original_df.at[index,'LOG_TEXT']
    address_found = re.search(address_regex,log_text)

    if address_found != None:
        parsed_address = clean_address(address_found)
        print(parsed_address.strip())
        original_df.at[index,'STR_DIRECCION_INCIDENTE'] = parsed_address.strip()
        return "Rebuilt"
    else:
        original_df.at[index,'STR_DIRECCION_INCIDENTE'] = 'ND'
        return "Not processed"

def clean_address(address_found):
    exclude_char_list = ['~','/','*','(',')']
    one_occurrence = address_found.group().split(',,,')[0].replace(',',' ')
    final_address = one_occurrence
    
    for char in exclude_char_list:
        if char in one_occurrence:
            final_address = final_address.split(char)[0]
            
    numbers_in_substring = re.sub('[^0-9]','', final_address)
    numbers_proportion = len(numbers_in_substring)/len(final_address)
    
    if numbers_proportion < 0.2:
        final_address = 'ND'
    
    return final_address


## Implement rebuild methods

### Rebuild location through address

In [ ]:
data_location = '/home/combios/Documents/amreyesp/clean_nuse_data/rebuild_address_nuse_14012020.csv'
df_input = pd.read_csv(data_location,delimiter=",")

In [4]:
pd.DataFrame({"Tipo de dato":df_input.dtypes.values,
              "Celdas con valor '-'":(df_input == '-').sum().values,
              "Celdas con valor ''":(df_input == '').sum().values,
              "Celdas con valor ' '":(df_input == ' ').sum().values,
              "Celdas vacías": df_input.isna().sum().values},
             index=df_input.columns)

/home/combios/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Tipo de dato,Celdas con valor '-',Celdas con valor '',Celdas con valor ' ',Celdas vacías
STR_NUMERO_INTERNO,object,0,0,0,0
FECHA,object,0,0,0,0
HORA,int64,0,0,0,0
ANIO,int64,0,0,0,0
MES,int64,0,0,0,0
COD_LOCALIDAD,object,6,0,0,0
LOCALIDAD,object,6,0,0,0
COD_UPZ,object,51950,0,0,0
UPZ,object,51950,0,0,0
COD_SEC_CATAST,object,51950,0,0,0


# Process small dataset to enhance web scraping performance

## Get original idx list to process

In [11]:
data_location = '/home/combios/Documents/amreyesp/clean_nuse_data/rebuild_address_nuse_14012020.csv'
df_input = pd.read_csv(data_location,delimiter=",")

/home/combios/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
df1 = df_input.loc[df_input['COD_UPZ'] == '-']
df2 = df_input.loc[df_input['UPZ'] == '-']
df3 = df_input.loc[df_input['COD_SEC_CATAST'] == '-']
df4 = df_input.loc[df_input['SEC_CATASTRAL'] == '-']
df5 = df_input.loc[df_input['COD_BARRIO'] == '-']
df6 = df_input.loc[df_input['BARRIO'] == '-']

In [13]:
df1.equals(df2) and df1.equals(df3) and df1.equals(df4) and df1.equals(df5) and df1.equals(df6)

True

In [14]:
#Try to rebuild 'sector catastral', 'UPZ', 'localidad', 'latitud', 'longitud' through address
df_empty_locations_with_address = df1.loc[df1['STR_DIRECCION_INCIDENTE'] != 'ND']
list_idx_rebuild_location = list(df_empty_locations_with_address.index.values)
len(list_idx_rebuild_location)

48447

## Iterate over small datasets

In [19]:
import time
from datetime import datetime

In [22]:
data_location = '/home/combios/Documents/amreyesp/clean_nuse_data/rebuild_locations_nuse_idx_39000_to_39999_24012020.csv'
df_input = pd.read_csv(data_location,delimiter=",")

In [23]:
df_output = df_input
idx_error_loading_address = []
num_reg_by_iter = 1000
registers_to_process = 8*num_reg_by_iter
rebuilt_registers = 0
registers_not_processed = 0
other_condition_counter = 0

file_path = '/home/combios/Documents/amreyesp/'

for i in range(40, 48):
    first_reg = i*num_reg_by_iter
    last_reg = ((i+1)*num_reg_by_iter)-1
    print(first_reg,' to ', last_reg)
    current_list_idx_rebuild = list_idx_rebuild_location[first_reg:last_reg]
    
    #Rebuild 'sector catastral', 'UPZ', 'localidad', 'latitud', 'longitud' using web scraping  
    url='https://mapas.bogota.gov.co'
    driver = ws_address.web_scrap_page(url)

    for index in current_list_idx_rebuild:
        state = rebuild_location_in_nuse(df_output, index, driver)
        if state == "Rebuilt":
            rebuilt_registers += 1
        elif state == "Not processed":
            registers_not_processed += 1
        elif state == "Error loading address":
            idx_error_loading_address.append(index)
        else:
            other_condition_counter += 1

        print('Rebuilt registers: ',rebuilt_registers,'/',registers_to_process)
        print('Registers not processed: ',registers_not_processed, '/', registers_to_process)
        
    time.sleep(30)
    # Save df output
    string_date = datetime.today().strftime('%d%m%Y')
    str_reg = str(first_reg) +'_to_'+ str(last_reg)
    file_name = "'" + file_path + 'rebuild_locations_nuse_idx_' + str_reg + '_' + string_date + ".csv'"
    df_output.to_csv(eval('r'+file_name),index=None)
    
    time.sleep(30)
    driver.quit()

40000  to  40999
KR 67 57 18
{'Dirección ingresada': 'KR 67 57 18', 'Dirección encontrada': 'KR 67 60B 26', 'Tipo dirección': 'Aproximación por malla', 'Código postal': '111911', 'Sector catastral': 'MADELENA', 'UPZ': 'ARBORIZADORA (UPZ65)', 'Localidad': 'CIUDAD BOLÍVAR', 'Latitud': '4.59178624200007', 'Longitud': '-74.1573715989999'}
Rebuilt registers:  1 / 8000
Registers not processed:  0 / 8000
CL 90 91 22
{'Dirección ingresada': 'CL 90 91 22', 'Dirección encontrada': 'AC 90 91 22', 'Tipo dirección': 'Asignada por Catastro', 'Código postal': '111021', 'Sector catastral': 'CIUDAD BACHUE', 'UPZ': 'MINUTO DE DIOS (UPZ29)', 'Localidad': 'ENGATIVÁ', 'Latitud': '4.71334272700005', 'Longitud': '-74.0978256469999'}
Rebuilt registers:  2 / 8000
Registers not processed:  0 / 8000
CL 95 60A
{'Dirección ingresada': 'CL 95 60A', 'Dirección encontrada': 'CL 94B 60A 0', 'Tipo dirección': 'Aproximación por malla', 'Código postal': '111211', 'Sector catastral': 'RIONEGRO', 'UPZ': 'LOS ANDES (UPZ21)'

Rebuilt registers:  21 / 8000
Registers not processed:  3 / 8000
CL 2 82 58
{'Dirección ingresada': 'CL 2 82 58', 'Dirección encontrada': 'CL 2 82 58', 'Tipo dirección': 'Asignada por Catastro', 'Código postal': '110851', 'Sector catastral': 'MARIA PAZ', 'UPZ': 'CORABASTOS (UPZ80)', 'Localidad': 'KENNEDY', 'Latitud': '4.63782553000004', 'Longitud': '-74.1567443319999'}
Rebuilt registers:  22 / 8000
Registers not processed:  3 / 8000
KR 12B 8 SUR
{'Dirección ingresada': 'KR 12B 8 SUR', 'Dirección encontrada': 'KR 12B 8 00 S', 'Tipo dirección': 'Asignada por Catastro', 'Código postal': '111511', 'Sector catastral': 'CIUDAD BERNA', 'UPZ': 'CIUDAD JARDIN (UPZ35)', 'Localidad': 'ANTONIO NARIÑO', 'Latitud': '4.58618765200009', 'Longitud': '-74.0915820939999'}
Rebuilt registers:  23 / 8000
Registers not processed:  3 / 8000
KR 28 7 85
{'Dirección ingresada': 'KR 28 7 85', 'Dirección encontrada': 'KR 28 7 83', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '111411', 'Sector catas

{'Dirección ingresada': 'KR 14A 83 84', 'Dirección encontrada': 'KR 14A 83 76', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '110221', 'Sector catastral': 'EL RETIRO', 'UPZ': 'CHICO LAGO (UPZ97)', 'Localidad': 'CHAPINERO', 'Latitud': '4.66906802100004', 'Longitud': '-74.0550899449999'}
Rebuilt registers:  43 / 8000
Registers not processed:  5 / 8000
KR 10A 48 SUR 28
{'Dirección ingresada': 'KR 10A 48 SUR 28', 'Dirección encontrada': 'KR 10A 48 28 S', 'Tipo dirección': 'Asignada por Catastro', 'Código postal': '111831', 'Sector catastral': 'ARBOLEDA SUR', 'UPZ': 'MARRUECOS (UPZ54)', 'Localidad': 'RAFAEL URIBE URIBE', 'Latitud': '4.56234386400007', 'Longitud': '-74.1138918069999'}
Rebuilt registers:  44 / 8000
Registers not processed:  5 / 8000
DG 32A BIS B SUR 12D 26
{'Dirección ingresada': 'DG 32A BIS B SUR 12D 26', 'Dirección encontrada': 'DG 32A BIS B S 12D 26', 'Tipo dirección': 'Asignada por Catastro', 'Código postal': '111821', 'Sector catastral': 'GRANJAS DE SANTA

{'Dirección ingresada': 'KR 15 108 90', 'Dirección encontrada': 'AK 15 108 50', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '110111', 'Sector catastral': 'MOLINOS NORTE', 'UPZ': 'SANTA BARBARA (UPZ16)', 'Localidad': 'USAQUÉN', 'Latitud': '4.69234380800009', 'Longitud': '-74.0446814809999'}
Rebuilt registers:  64 / 8000
Registers not processed:  6 / 8000
KR 32 64 SUR 52
{'Dirección ingresada': 'KR 32 64 SUR 52', 'Dirección encontrada': 'KR 32 64 40 S', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '111941', 'Sector catastral': 'CANDELARIA LA NUEVA', 'UPZ': 'SAN FRANCISCO (UPZ66)', 'Localidad': 'CIUDAD BOLÍVAR', 'Latitud': '4.56887431600006', 'Longitud': '-74.1503555039999'}
Rebuilt registers:  65 / 8000
Registers not processed:  6 / 8000
CL 133C 155 03
{'Dirección ingresada': 'CL 133C 155 03', 'Dirección encontrada': 'CL 133 154A 03', 'Tipo dirección': 'Aproximación por malla', 'Código postal': '111151', 'Sector catastral': 'SANTA CECILIA', 'UPZ': 'TIBABU

{'Dirección ingresada': 'CL 143 8 47', 'Dirección encontrada': 'CL 143 9 55', 'Tipo dirección': 'Aproximación por malla', 'Código postal': '110121', 'Sector catastral': 'CEDRO NARVAEZ', 'UPZ': 'LOS CEDROS (UPZ13)', 'Localidad': 'USAQUÉN', 'Latitud': '4.72093438600007', 'Longitud': '-74.0333111659999'}
Rebuilt registers:  84 / 8000
Registers not processed:  8 / 8000
CL 87A SUR 12 ESTE
Rebuilt registers:  84 / 8000
Registers not processed:  9 / 8000
CL 39 SUR 52A 10
{'Dirección ingresada': 'CL 39 SUR 52A 10', 'Dirección encontrada': 'CL 39 S 52A 10', 'Tipo dirección': 'Asignada por Catastro', 'Código postal': '111621', 'Sector catastral': 'ALQUERIA', 'UPZ': 'MUZU (UPZ41)', 'Localidad': 'PUENTE ARANDA', 'Latitud': '4.60067543400004', 'Longitud': '-74.131576273'}
Rebuilt registers:  85 / 8000
Registers not processed:  9 / 8000
CL 48P SUR 1A 42
{'Dirección ingresada': 'CL 48P SUR 1A 42', 'Dirección encontrada': 'CL 48P S 1A 42', 'Tipo dirección': 'Asignada por Catastro', 'Código postal': '1

...

{'Dirección ingresada': 'CL 59 SUR 22B 58', 'Dirección encontrada': 'CL 59 S 22B 58', 'Tipo dirección': 'Asignada por Catastro', 'Código postal': '111911', 'Sector catastral': 'EL CHIRCAL SUR', 'UPZ': 'ARBORIZADORA (UPZ65)', 'Localidad': 'CIUDAD BOLÍVAR', 'Latitud': '4.57022937900007', 'Longitud': '-74.1442470279999'}
Rebuilt registers:  6784 / 8000
Registers not processed:  1079 / 8000
KR 120 64C 08
{'Dirección ingresada': 'KR 120 64C 08', 'Dirección encontrada': 'KR 120 64C 08', 'Tipo dirección': 'Asignada por Catastro', 'Código postal': '111031', 'Sector catastral': 'ENGATIVA ZONA URBANA', 'UPZ': 'ENGATIVA (UPZ74)', 'Localidad': 'ENGATIVÁ', 'Latitud': '4.71385668900007', 'Longitud': '-74.1403585559999'}
Rebuilt registers:  6785 / 8000
Registers not processed:  1079 / 8000
KR 112 135
{'Dirección ingresada': 'KR 112 135', 'Dirección encontrada': 'KR 112 135B 0', 'Tipo dirección': 'Aproximación por malla', 'Código postal': '111141', 'Sector catastral': 'VILLA MARIA', 'UPZ': 'EL RINCON 

{'Dirección ingresada': 'KR 18Q BIS 78 SUR 68', 'Dirección encontrada': 'KR 18Q 78 S 68', 'Tipo dirección': 'Aproximación por malla', 'Código postal': '111961', 'Sector catastral': 'NACIONES UNIDAS', 'UPZ': 'EL TESORO (UPZ68)', 'Localidad': 'CIUDAD BOLÍVAR', 'Latitud': '4.53860946260183', 'Longitud': '-74.1505554047737'}
Rebuilt registers:  6804 / 8000
Registers not processed:  1082 / 8000
CL 67B SUR 18 24
{'Dirección ingresada': 'CL 67B SUR 18 24', 'Dirección encontrada': 'CL 67B S 18 36', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '111961', 'Sector catastral': 'LUCERO DEL SUR', 'UPZ': 'LUCERO (UPZ67)', 'Localidad': 'CIUDAD BOLÍVAR', 'Latitud': '4.55046000900006', 'Longitud': '-74.1440234809999'}
Rebuilt registers:  6805 / 8000
Registers not processed:  1082 / 8000
KR 18K 72 SUR 29
{'Dirección ingresada': 'KR 18K 72 SUR 29', 'Dirección encontrada': 'TV 18I BIS A 72 S 29', 'Tipo dirección': 'Aproximación por malla', 'Código postal': '111961', 'Sector catastral': 'NACI

{'Dirección ingresada': 'KR 75 63 44', 'Dirección encontrada': 'KR 75 63C 44', 'Tipo dirección': 'Aproximación por malla', 'Código postal': '111071', 'Sector catastral': 'EL ENCANTO', 'UPZ': 'SANTA CECILIA (UPZ31)', 'Localidad': 'ENGATIVÁ', 'Latitud': '4.67888603429999', 'Longitud': '-74.1066880994225'}
Rebuilt registers:  6825 / 8000
Registers not processed:  1083 / 8000
KR 86 82 89
{'Dirección ingresada': 'KR 86 82 89', 'Dirección encontrada': 'AK 86 82 62', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '111021', 'Sector catastral': 'PARIS GAITAN', 'UPZ': 'MINUTO DE DIOS (UPZ29)', 'Localidad': 'ENGATIVÁ', 'Latitud': '4.70444614900009', 'Longitud': '-74.0994718689999'}
Rebuilt registers:  6826 / 8000
Registers not processed:  1083 / 8000
CL 37 SUR 72D
{'Dirección ingresada': 'CL 37 SUR 72D', 'Dirección encontrada': 'DG 37 S 72D 19', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '110841', 'Sector catastral': 'PROVIVIENDA OCCIDENTAL', 'UPZ': 'CARVAJAL (UPZ4

{'Dirección ingresada': 'KR 11 9 23', 'Dirección encontrada': 'KR 11 9 07', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '110321', 'Sector catastral': 'SANTA INES', 'UPZ': 'LAS NIEVES (UPZ93)', 'Localidad': 'SANTA FE', 'Latitud': '4.59887084000007', 'Longitud': '-74.0799356639999'}
Rebuilt registers:  6846 / 8000
Registers not processed:  1084 / 8000
KR 11 11 21
{'Dirección ingresada': 'KR 11 11 21', 'Dirección encontrada': 'KR 11 11 19', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '110321', 'Sector catastral': 'SANTA INES', 'UPZ': 'LAS NIEVES (UPZ93)', 'Localidad': 'SANTA FE', 'Latitud': '4.60077174500009', 'Longitud': '-74.0786875329999'}
Rebuilt registers:  6847 / 8000
Registers not processed:  1084 / 8000
KR 120B 73 14
{'Dirección ingresada': 'KR 120B 73 14', 'Dirección encontrada': 'KR 120B 73 02', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '111031', 'Sector catastral': 'VILLAS DE ALCALA', 'UPZ': 'GARCES NAVAS (UPZ73)', 'Localidad

{'Dirección ingresada': 'CL 1D 5A 37', 'Dirección encontrada': 'CL 1D 5A 37', 'Tipo dirección': 'Asignada por Catastro', 'Código postal': '110321', 'Sector catastral': 'LAS CRUCES', 'UPZ': 'LAS CRUCES (UPZ95)', 'Localidad': 'SANTA FE', 'Latitud': '4.58719568300006', 'Longitud': '-74.0795479129999'}
Rebuilt registers:  6867 / 8000
Registers not processed:  1085 / 8000
TV 3 ESTE 24 SUR 22
Rebuilt registers:  6867 / 8000
Registers not processed:  1086 / 8000
KR 12J 32 SUR 11
{'Dirección ingresada': 'KR 12J 32 SUR 11', 'Dirección encontrada': 'KR 12J 32 10 S', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '111821', 'Sector catastral': 'HOSPITAL SAN CARLOS', 'UPZ': 'SAN JOSE (UPZ36)', 'Localidad': 'RAFAEL URIBE URIBE', 'Latitud': '4.57337452800004', 'Longitud': '-74.1092548319999'}
Rebuilt registers:  6868 / 8000
Registers not processed:  1086 / 8000
KR 88H 40 SUR 35
{'Dirección ingresada': 'KR 88H 40 SUR 35', 'Dirección encontrada': 'KR 88H 40 33 S', 'Tipo dirección': 'Aprox

In [24]:
print(rebuilt_registers)
print(registers_not_processed)
print(other_condition_counter)

6886
1087
0


In [26]:
#save lists
MyFile=open('/home/combios/Documents/amreyesp/idx_error_loading_address_29012020.txt','w')
MyList=map(lambda x: str(x)+', ', idx_error_loading_address)
MyFile.writelines(MyList)
MyFile.close()

In [25]:
df_output.to_csv(r'/home/combios/Documents/amreyesp/rebuild_locations_nuse_idx_39kto48k_29012020.csv',index=None)

In [19]:
#iterate over idx_error_loading_address to rebuild location if necesary